In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
datapath =
df.head()

SyntaxError: unterminated string literal (detected at line 1) (ipython-input-1821066950.py, line 1)